#  〇、千言数据集：文本相似度比赛简介

文本相似度旨在识别两段文本在语义上是否相似。文本相似度在自然语言处理领域是一个重要研究方向，同时在信息检索、新闻推荐、智能客服等领域都发挥重要作用，具有很高的商业价值。

[ 文本相似度：https://aistudio.baidu.com/aistudio/competition/detail/45](https://aistudio.baidu.com/aistudio/competition/detail/45)

目前学术界的一些公开中文文本相似度数据集，在相关论文的支撑下对现有的公开文本相似度模型进行了较全面的评估，具有较高权威性。因此，本开源项目收集了这些权威的数据集，期望对模型效果进行综合的评价，旨在为研究人员和开发者提供学术和技术交流的平台，进一步提升文本相似度的研究水平，推动文本相似度在自然语言处理领域的应用和发展。

## 1.数据集
本次评测的文本相似度数据集包括公开的三个文本相似度数据集，分别为哈尔滨工业大学（深圳）的 LCQMC [1] 和 BQ Coupus [2]，以及谷歌的 PAWS-X（中文） [3]。各数据集的简介如下：

### a) LCQMC
LCQMC（A Large-scale Chinese Question Matching Corpus）, 百度知道领域的中文问题匹配数据集，目的是为了解决在中文领域大规模问题匹配数据集的缺失。该数据集从百度知道不同领域的用户问题中抽取构建数据。

### b) BQ Corpus
BQ Corpus（Bank Question Corpus）, 银行金融领域的问题匹配数据，包括了从一年的线上银行系统日志里抽取的问题pair对，是目前最大的银行领域问题匹配数据。

### c) PAWS-X (中文)
PAWS (Paraphrase Adversaries from Word Scrambling)，谷歌发布的包含 7 种语言释义对的数据集，包括PAWS（英语） 与 PAWS-X（多语）。数据集里包含了释义对和非释义对，即识别一对句子是否具有相同的释义（含义），特点是具有高度重叠词汇，对于进一步提升模型对于强负例的判断很有帮助。

各个数据集的任务均一致，即判断两段文本在语义上是否相似的二分类任务。以LCQMC为例：

## 2.提交方式
```
# 文本相似度任务                                                                 
index   prediction                                                              
0   1                                                                           
1   0                                                                           
2   1    
```

# 二、思路

* 1.搭建网络
* 2.更换数据集
* 3.针对不同数据集Finetune 生成不同模型
* 4.用不同模型进行预测

# 三、环境准备
## 1.包引入

In [ ]:
!python -m pip install --upgrade paddlenlp==2.0.2 -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 430kB 18.9MB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


In [ ]:
import time
import os
import numpy as np

import paddle
import paddle.nn.functional as F
from paddlenlp.datasets import load_dataset
import paddlenlp

# 一键加载 Lcqmc 的训练集、验证集
train_ds, dev_ds = load_dataset("lcqmc", splits=["train", "dev"])

100%|██████████| 6827/6827 [00:00<00:00, 51482.40it/s]


## 2.数据下载

In [ ]:
# paddlenlp 会自动下载 lcqmc 数据集解压到 "${HOME}/.paddlenlp/datasets/LCQMC/lcqmc/lcqmc/" 目录下
! ls ${HOME}/.paddlenlp/datasets/LCQMC/lcqmc/lcqmc
print(paddlenlp.__version__)

dev.tsv  License.pdf  test.tsv	train.tsv  User_Agreement.pdf
2.0.2


## 3.数据查看

In [ ]:
# 输出训练集的前 20 条样本
for idx, example in enumerate(train_ds):
    if idx <= 20:
        print(example)

{'query': '喜欢打篮球的男生喜欢什么样的女生', 'title': '爱打篮球的男生喜欢什么样的女生', 'label': 1}
{'query': '我手机丢了，我想换个手机', 'title': '我想买个新手机，求推荐', 'label': 1}
{'query': '大家觉得她好看吗', 'title': '大家觉得跑男好看吗？', 'label': 0}
{'query': '求秋色之空漫画全集', 'title': '求秋色之空全集漫画', 'label': 1}
{'query': '晚上睡觉带着耳机听音乐有什么害处吗？', 'title': '孕妇可以戴耳机听音乐吗?', 'label': 0}
{'query': '学日语软件手机上的', 'title': '手机学日语的软件', 'label': 1}
{'query': '打印机和电脑怎样连接，该如何设置', 'title': '如何把带无线的电脑连接到打印机上', 'label': 0}
{'query': '侠盗飞车罪恶都市怎样改车', 'title': '侠盗飞车罪恶都市怎么改车', 'label': 1}
{'query': '什么花一年四季都开', 'title': '什么花一年四季都是开的', 'label': 1}
{'query': '看图猜一电影名', 'title': '看图猜电影！', 'label': 1}
{'query': '这上面写的是什么？', 'title': '胃上面是什么', 'label': 0}
{'query': '建议您重新注册，辛苦您了。', 'title': '可以的，您注销成功后，可以重新注册的，辛苦您了。', 'label': 0}
{'query': '小草有什么的特点,可以象征什么?', 'title': '小草有什么特点可以象征什么', 'label': 1}
{'query': '校验失败了，', 'title': '您好，您还是访客状态呢', 'label': 0}
{'query': '尼玛什么意思', 'title': '尼玛啊是什么意思？', 'label': 0}
{'query': '自找苦吃的地方是哪儿？', 'title': '自找苦吃的地方是哪儿', 'label': 1}
{'query': '尾号4位多

## 4.数据预处理
通过 paddlenlp 加载进来的 LCQMC 数据集是原始的明文数据集，这部分我们来实现组 batch、tokenize 等预处理逻辑，将原始明文数据转换成网络训练的输入数据

### 4.1定义样本转换函数

In [ ]:
# 因为是基于预训练模型 ERNIE-Gram 来进行，所以需要首先加载 ERNIE-Gram 的 tokenizer，
# 后续样本转换函数基于 tokenizer 对文本进行切分

# tokenizer = paddlenlp.transformers.ErnieGramTokenizer.from_pretrained('ernie-gram-zh')

tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained('ernie-1.0')
# tokenizer = paddlenlp.transformers.NeZhaTokenizer.from_pretrained('nezha-large-wwm-chinese')

[2021-06-15 13:39:36,356] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt
100%|██████████| 90/90 [00:00<00:00, 3296.23it/s]


In [ ]:
# 将 1 条明文数据的 query、title 拼接起来，根据预训练模型的 tokenizer 将明文转换为 ID 数据
# 返回 input_ids 和 token_type_ids

def convert_example(example, tokenizer, max_seq_length=512, is_test=False):

    query, title = example["query"], example["title"]

    encoded_inputs = tokenizer(
        text=query, text_pair=title, max_seq_len=max_seq_length)

    input_ids = encoded_inputs["input_ids"]
    token_type_ids = encoded_inputs["token_type_ids"]

    if not is_test:
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    # 在预测或者评估阶段，不返回 label 字段
    else:
        return input_ids, token_type_ids

In [ ]:
### 对训练集的第 1 条数据进行转换
input_ids, token_type_ids, label = convert_example(train_ds[0], tokenizer)

In [8]:
print(input_ids)

[1, 692, 811, 445, 2001, 497, 5, 654, 21, 692, 811, 614, 356, 314, 5, 291, 21, 2, 329, 445, 2001, 497, 5, 654, 21, 692, 811, 614, 356, 314, 5, 291, 21, 2]


In [9]:
print(token_type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
# 为了后续方便使用，我们给 convert_example 赋予一些默认参数
from functools import partial

# 训练集和验证集的样本转换函数
trans_func = partial(
    convert_example,
    tokenizer=tokenizer,
    max_seq_length=512)

### 4.2 组装 Batch 数据 & Padding
上一小节，我们完成了对单条样本的转换，本节我们需要将样本组合成 Batch 数据，对于不等长的数据还需要进行 Padding 操作，便于 GPU 训练。

PaddleNLP 提供了许多关于 NLP 任务中构建有效的数据 pipeline 的常用 API

In [11]:
from paddlenlp.data import Stack, Pad, Tuple
# 我们的训练数据会返回 input_ids, token_type_ids, labels 3 个字段
# 因此针对这 3 个字段需要分别定义 3 个组 batch 操作
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack(dtype="int64")  # label
): [data for data in fn(samples)]

### 4.3 定义 Dataloader
下面我们基于组 batchify_fn 函数和样本转换函数 trans_func 来构造训练集的 DataLoader, 支持多卡训练

In [12]:
# 定义分布式 Sampler: 自动对训练数据进行切分，支持多卡并行训练
batch_sampler = paddle.io.DistributedBatchSampler(train_ds, batch_size=200, shuffle=True) # batch_size=32

# 基于 train_ds 定义 train_data_loader
# 因为我们使用了分布式的 DistributedBatchSampler, train_data_loader 会自动对训练数据进行切分
train_data_loader = paddle.io.DataLoader(
        dataset=train_ds.map(trans_func),
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)

# 针对验证集数据加载，我们使用单卡进行评估，所以采用 paddle.io.BatchSampler 即可
# 定义 dev_data_loader
batch_sampler = paddle.io.BatchSampler(dev_ds, batch_size=200, shuffle=False)
dev_data_loader = paddle.io.DataLoader(
        dataset=dev_ds.map(trans_func),
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)

# 四、模型训练
## 1.模型搭建
自从 2018 年 10 月以来，NLP 个领域的任务都通过 Pretrain + Finetune 的模式相比传统 DNN 方法在效果上取得了显著的提升，本节我们以百度开源的预训练模型 ERNIE-Gram 为基础模型，在此之上构建 Point-wise 语义匹配网络。

In [13]:
import paddle.nn as nn

# 我们基于 ERNIE-Gram 模型结构搭建 Point-wise 语义匹配网络
# 所以此处先定义 ERNIE-Gram 的 pretrained_model
# pretrained_model = paddlenlp.transformers.ErnieGramModel.from_pretrained('ernie-gram-zh')
pretrained_model = paddlenlp.transformers.ErnieModel.from_pretrained('ernie-1.0')
# pretrained_model = paddlenlp.transformers.NeZhaModel.from_pretrained('nezha-large-wwm-chinese')



class PointwiseMatching(nn.Layer):
   
    # 此处的 pretained_model 在本例中会被 ERNIE-Gram 预训练模型初始化
    def __init__(self, pretrained_model, dropout=None):
        super().__init__()
        self.ptm = pretrained_model
        self.dropout = nn.Dropout(dropout if dropout is not None else 0.07) #0.1

        # 语义匹配任务: 相似、不相似 2 分类任务
        self.classifier = nn.Linear(self.ptm.config["hidden_size"], 2)

    def forward(self,
                input_ids,
                token_type_ids=None,
                position_ids=None,
                attention_mask=None):

        # 此处的 Input_ids 由两条文本的 token ids 拼接而成
        # token_type_ids 表示两段文本的类型编码
        # 返回的 cls_embedding 就表示这两段文本经过模型的计算之后而得到的语义表示向量
        _, cls_embedding = self.ptm(input_ids, token_type_ids, position_ids,
                                    attention_mask)


        cls_embedding = self.dropout(cls_embedding)

        # 基于文本对的语义表示向量进行 2 分类任务
        logits = self.classifier(cls_embedding)
        probs = F.softmax(logits)

        return probs

# 定义 Point-wise 语义匹配网络
model = PointwiseMatching(pretrained_model)

[2021-06-15 13:39:36,533] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie/ernie_v1_chn_base.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-1.0
[2021-06-15 13:39:36,536] [    INFO] - Downloading ernie_v1_chn_base.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/ernie_v1_chn_base.pdparams
100%|██████████| 392507/392507 [00:07<00:00, 53183.89it/s]
[2021-06-15 13:39:50,553] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']


## 2. 模型训练（引入visual dl）

In [14]:
from paddlenlp.transformers import LinearDecayWithWarmup

epochs = 100
num_training_steps = len(train_data_loader) * epochs

# 定义 learning_rate_scheduler，负责在训练过程中对 lr 进行调度
lr_scheduler = LinearDecayWithWarmup(5E-5, num_training_steps, 0.0)

# Generate parameter names needed to perform weight decay.
# All bias and LayerNorm parameters are excluded.
decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 定义 Optimizer
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=0.0,
    apply_decay_param_fun=lambda x: x in decay_params)

# 采用交叉熵 损失函数
criterion = paddle.nn.loss.CrossEntropyLoss()

# 评估的时候采用准确率指标
metric = paddle.metric.Accuracy()

In [15]:
# 加入日志显示
from visualdl import LogWriter

writer = LogWriter("./log")

In [16]:
# 因为训练过程中同时要在验证集进行模型评估，因此我们先定义评估函数

@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader, phase="dev"):
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch
        # probs = model(input_ids=input_ids, token_type_ids=token_type_ids)
        probs = model(input_ids=input_ids, token_type_ids=token_type_ids)
        loss = criterion(probs, labels)
        losses.append(loss.numpy())
        correct = metric.compute(probs, labels)
        metric.update(correct)
        accu = metric.accumulate()
        tmpacc=accu
    print("eval {} loss: {:.5}, accu: {:.5}".format(phase,
                                                    np.mean(losses), accu))
    model.train()
    metric.reset()
    return np.mean(losses), accu

In [17]:
# 接下来，开始正式训练模型

global_step = 0
tic_train = time.time()
best_val_acc=0
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):

        input_ids, token_type_ids, labels = batch
        probs = model(input_ids=input_ids, token_type_ids=token_type_ids)
        loss = criterion(probs, labels)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        global_step += 1
        
        # 每间隔 10 step 输出训练指标
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()


        # 每间隔 200 step 在验证集和测试集上进行评估
        if global_step %100 == 0:
            eval_loss, eval_accu=evaluate(model, criterion, metric, dev_data_loader, "dev")
            # 加入eval日志显示
            writer.add_scalar(tag="eval/loss", step=global_step, value=eval_loss)
            writer.add_scalar(tag="eval/acc", step=global_step, value=eval_accu)    
        
        # 加入train日志显示
            writer.add_scalar(tag="train/loss", step=global_step, value=loss)
            writer.add_scalar(tag="train/acc", step=global_step, value=acc)
            
            save_dir = "checkpoint"
            # 加入保存       
            if eval_accu>best_val_acc:
                if not os.path.exists(save_dir):
                    os.mkdir(save_dir)
                best_val_acc=eval_accu
                print(f"模型保存在 {global_step} 步， 最佳eval准确度为{best_val_acc:.8f}！")
                save_param_path = os.path.join(save_dir, 'best_model.pdparams')
                paddle.save(model.state_dict(), save_param_path)
                fh = open('checkpoint/best_model.txt', 'w', encoding='utf-8')
                fh.write(f"模型保存在 {global_step} 步， 最佳eval准确度为{best_val_acc:.8f}！")
                fh.close()

global step 10, epoch: 1, batch: 10, loss: 0.55927, accu: 0.61200, speed: 1.53 step/s
global step 20, epoch: 1, batch: 20, loss: 0.45002, accu: 0.73675, speed: 1.55 step/s
global step 30, epoch: 1, batch: 30, loss: 0.40220, accu: 0.78633, speed: 1.54 step/s
global step 40, epoch: 1, batch: 40, loss: 0.43648, accu: 0.80875, speed: 1.51 step/s
global step 50, epoch: 1, batch: 50, loss: 0.39423, accu: 0.82470, speed: 1.46 step/s
global step 60, epoch: 1, batch: 60, loss: 0.43224, accu: 0.83667, speed: 1.54 step/s
global step 70, epoch: 1, batch: 70, loss: 0.39952, accu: 0.84457, speed: 1.54 step/s
global step 80, epoch: 1, batch: 80, loss: 0.38990, accu: 0.85175, speed: 1.51 step/s
global step 90, epoch: 1, batch: 90, loss: 0.39998, accu: 0.85694, speed: 1.55 step/s
global step 100, epoch: 1, batch: 100, loss: 0.40286, accu: 0.86130, speed: 1.55 step/s
eval dev loss: 0.47756, accu: 0.82402
模型保存在 100 步， 最佳eval准确度为0.82401727！
global step 110, epoch: 1, batch: 110, loss: 0.42226, accu: 0.884

KeyboardInterrupt: 

In [18]:
# 训练结束后，存储模型参数
# save_dir = os.path.join("checkpoint", "model_%d" % global_step)
# os.makedirs(save_dir)
tokenizer.save_pretrained(save_dir)
# save_param_path = os.path.join(save_dir, 'model_state.pdparams')
# paddle.save(model.state_dict(), save_param_path)
# tokenizer.save_pretrained(save_dir)
break!!!

SyntaxError: invalid syntax (<ipython-input-18-fb6bd3295eff>, line 8)

# 五、模型预测
接下来我们使用已经训练好的语义匹配模型对一些预测数据进行预测。待预测数据为每行都是文本对的 tsv 文件，我们使用 Lcqmc 数据集的测试集作为我们的预测数据，进行预测并提交预测结果到 千言文本相似度竞赛

下载我们已经训练好的语义匹配模型, 并解压

In [ ]:
break

In [19]:
# 下载我们基于 Lcqmc 事先训练好的语义匹配模型并解压
! wget https://paddlenlp.bj.bcebos.com/models/text_matching/ernie_gram_zh_pointwise_matching_model.tar
! tar -xvf ernie_gram_zh_pointwise_matching_model.tar

--2021-06-15 14:41:34--  https://paddlenlp.bj.bcebos.com/models/text_matching/ernie_gram_zh_pointwise_matching_model.tar
Resolving paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c00:6c21:10ad:0:ff:b00e:67d
Connecting to paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 597667840 (570M) [application/x-tar]
Saving to: ‘ernie_gram_zh_pointwise_matching_model.tar.1’

ernie_gram_zh_point 100%[===================>] 569.98M  54.6MB/s    in 12s     

2021-06-15 14:41:46 (45.7 MB/s) - ‘ernie_gram_zh_pointwise_matching_model.tar.1’ saved [597667840/597667840]

ernie_gram_zh_pointwise_matching_model/
ernie_gram_zh_pointwise_matching_model/model_state.pdparams
ernie_gram_zh_pointwise_matching_model/vocab.txt
ernie_gram_zh_pointwise_matching_model/tokenizer_config.json


In [ ]:
# 测试数据由 2 列文本构成 tab 分隔
# Lcqmc 默认下载到如下路径
! head -n10 "${HOME}/.paddlenlp/datasets/LCQMC/lcqmc/lcqmc/test.tsv"

## 1.定义预测函数

In [23]:
def predict(model, data_loader):
    
    batch_probs = []

    # 预测阶段打开 eval 模式，模型中的 dropout 等操作会关掉
    model.eval()

    with paddle.no_grad():
        for batch_data in data_loader:
            input_ids, token_type_ids = batch_data
            input_ids = paddle.to_tensor(input_ids)
            token_type_ids = paddle.to_tensor(token_type_ids)
            
            # 获取每个样本的预测概率: [batch_size, 2] 的矩阵
            batch_prob = model(
                input_ids=input_ids, token_type_ids=token_type_ids).numpy()

            batch_probs.append(batch_prob)
        batch_probs = np.concatenate(batch_probs, axis=0)

        return batch_probs

## 2.定义预测数据的 data_loader

In [ ]:
!head paws-x-zh/test.tsv
!head paws-x-zh/train.tsv

In [ ]:
!unzip data/data52714/lcqmc.zip -d lcqmc

In [21]:
# 预测数据的转换函数
# predict 数据没有 label, 因此 convert_exmaple 的 is_test 参数设为 True
trans_func = partial(
    convert_example,
    tokenizer=tokenizer,
    max_seq_length=512,
    is_test=True)

# 预测数据的组 batch 操作
# predict 数据只返回 input_ids 和 token_type_ids，因此只需要 2 个 Pad 对象作为 batchify_fn
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment_ids
): [data for data in fn(samples)]

# 加载预测数据
test_ds = load_dataset("lcqmc", splits='test')
# test_ds = load_dataset("lcqmc", data_files='paws-x-zh/test.tsv')
# test_ds = load_dataset("lcqmc", data_files='bq_corpus/test.tsv')

In [22]:
batch_sampler = paddle.io.BatchSampler(test_ds, batch_size=32, shuffle=False)

# 生成预测数据 data_loader
predict_data_loader =paddle.io.DataLoader(
        dataset=test_ds.map(trans_func),
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)

## 3. 定义预测模型

In [24]:
# pretrained_model = paddlenlp.transformers.ErnieGramModel.from_pretrained('ernie-gram-zh')
pretrained_model = paddlenlp.transformers.ErnieModel.from_pretrained('ernie-1.0')
model = PointwiseMatching(pretrained_model)

[2021-06-15 14:42:12,101] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-06-15 14:42:13,309] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']


## 4.加载已训练好的模型参数（此处也可以直接加载预训练模型进行预测，也可以加载最佳训练的checkpoint进行训练）


In [25]:
# 刚才下载的模型解压之后存储路径为 ./ernie_gram_zh_pointwise_matching_model/model_state.pdparams
# state_dict = paddle.load("./ernie_gram_zh_pointwise_matching_model/model_state.pdparams")

state_dict = paddle.load("checkpoint_lcqmc/best_model.pdparams")
# 刚才下载的模型解压之后存储路径为 ./pointwise_matching_model/ernie1.0_base_pointwise_matching.pdparams
# state_dict = paddle.load("pointwise_matching_model/ernie1.0_base_pointwise_matching.pdparams")
model.set_dict(state_dict)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for ptm.embeddings.word_embeddings.weight. ptm.embeddings.word_embeddings.weight receives a shape [18018, 768], but the expected shape is [18000, 768].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for ptm.embeddings.position_embeddings.weight. ptm.embeddings.position_embeddings.weight receives a shape [512, 768], but the expected shape is [513, 768].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))


## 5.开始预测

In [26]:
for idx, batch in enumerate(predict_data_loader):
    if idx < 1:
        print(batch)

[Tensor(shape=[32, 38], dtype=int64, place=CUDAPinnedPlace, stop_gradient=True,
       [[1   , 1022, 9   , ..., 0   , 0   , 0   ],
        [1   , 514 , 904 , ..., 0   , 0   , 0   ],
        [1   , 47  , 10  , ..., 0   , 0   , 0   ],
        ...,
        [1   , 733 , 404 , ..., 0   , 0   , 0   ],
        [1   , 134 , 170 , ..., 0   , 0   , 0   ],
        [1   , 379 , 3122, ..., 0   , 0   , 0   ]]), Tensor(shape=[32, 38], dtype=int64, place=CUDAPinnedPlace, stop_gradient=True,
       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]


In [27]:
# 执行预测函数
y_probs = predict(model, predict_data_loader)

# 根据预测概率获取预测 label
y_preds = np.argmax(y_probs, axis=1)

##  6.输出预测结果

In [28]:
# 我们按照千言文本相似度竞赛的提交格式将预测结果存储在 lcqmc.tsv 中，用来后续提交
# 同时将预测结果输出到终端，便于大家直观感受模型预测效果

# test_ds = load_dataset("lcqmc", splits=["test"])

# with open("lcqmc.tsv", 'w', encoding="utf-8") as f:
# with open("paws-x.tsv", 'w', encoding="utf-8") as f:
with open("lcqmc.tsv", 'w', encoding="utf-8") as f:
    f.write("index\tprediction\n")    
    for idx, y_pred in enumerate(y_preds):
        f.write("{}\t{}\n".format(idx, y_pred))
        print("{}\t{}\n".format(idx, y_pred))
        # text_pair = test_ds[idx]
        # text_pair["id"] = test_ds[idx]
        # text_pair["label"] = y_pred
        # print(text_pair)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1809	1

1810	1

1811	1

1812	1

1813	1

1814	0

1815	1

1816	1

1817	1

1818	1

1819	1

1820	0

1821	0

1822	1

1823	0

1824	1

1825	0

1826	1

1827	0

1828	0

1829	0

1830	1

1831	1

1832	0

1833	0

1834	0

1835	0

1836	1

1837	0

1838	1

1839	1

1840	0

1841	0

1842	1

1843	1

1844	0

1845	0

1846	0

1847	1

1848	0

1849	0

1850	1

1851	1

1852	0

1853	0

1854	1

1855	1

1856	0

1857	0

1858	1

1859	1

1860	0

1861	0

1862	1

1863	1

1864	1

1865	0

1866	0

1867	1

1868	1

1869	1

1870	0

1871	1

1872	1

1873	1

1874	0

1875	0

1876	0

1877	1

1878	0

1879	1

1880	1

1881	1

1882	1

1883	1

1884	1

1885	1

1886	1

1887	1

1888	1

1889	1

1890	1

1891	1

1892	0

1893	1

1894	0

1895	0

1896	1

1897	1

1898	1

1899	1

1900	1

1901	1

1902	0

1903	0


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




4900	1

4901	1

4902	1

4903	1

4904	1

4905	0

4906	1

4907	1

4908	1

4909	1

4910	1

4911	0

4912	0

4913	1

4914	0

4915	1

4916	0

4917	1

4918	0

4919	0

4920	1

4921	1

4922	0

4923	0

4924	0

4925	1

4926	0

4927	1

4928	0

4929	1

4930	0

4931	0

4932	0

4933	1

4934	1

4935	0

4936	1

4937	0

4938	1

4939	1

4940	0

4941	1

4942	1

4943	1

4944	1

4945	1

4946	1

4947	1

4948	0

4949	1

4950	0

4951	1

4952	1

4953	1

4954	0

4955	0

4956	1

4957	0

4958	1

4959	0

4960	1

4961	0

4962	0

4963	0

4964	0

4965	0

4966	1

4967	0

4968	1

4969	0

4970	1

4971	1

4972	1

4973	1

4974	1

4975	1

4976	0

4977	1

4978	0

4979	1

4980	1

4981	0

4982	1

4983	0

4984	1

4985	0

4986	1

4987	1

4988	1

4989	0

4990	0

4991	1

4992	0

4993	0

4994	0

4995	1

4996	0

4997	1

4998	0

4999	1

5000	1

5001	0

5002	1

5003	1

5004	1

5005	1

5006	1

5007	1

5008	1

5009	1

5010	1

5011	0

5012	1

5013	1

5014	0

5015	1

5016	1

5017	1

5018	1

5019	0

5020	0

5021	1

5022	0

5023	1

5024	1


# 六、BQ Corpus、PAWS-X (中文) 相似度预测流程

## 1.解压缩bq_corpus.zip、paws-x-zh.zip数据集
## 2.自定义数据集训练其他两类模型

代码如下自定义数据集，注意一个是数据集类型 **lcqmc**， 一个是文件夹所在位置，**splits** 提示所需返回的dataset
```
from paddlenlp.datasets import load_dataset
train_ds, dev_ds = load_dataset("lcqmc", data_files='bq_corpus/', splits=("train", "dev"))
```
## 3.替换预测数据集开始预测
指明数据格式和文件名
`test_ds = load_dataset("lcqmc", data_files='bq_corpus/test.tsv')`

In [ ]:
!unzip -qa data/data52714/bq_corpus.zip
!unzip -qa data/data52714/paws-x-zh.zip

## 4.提交 lcqmc 预测结果千言文本相似度竞赛
千言文本相似度竞赛一共有 3 个数据集: lcqmc、bq_corpus、paws-x, 我们刚才生成了 lcqmc 的预测结果 lcqmc.tsv, 同时我们在项目内提供了 bq_corpus、paw-x 数据集的空预测结果，我们将这 3 个文件打包提交到千言文本相似度竞赛，即可看到自己的模型在 Lcqmc 数据集上的竞赛成绩。

In [ ]:
# 打包预测结果
!zip submit.zip lcqmc.tsv paws-x.tsv bq_corpus.tsv

## 5.跑个demo看看
有时间思考思考跑起来啊啊啊

![](https://ai-studio-static-online.cdn.bcebos.com/f262f80927c14919b2ed2017d7c420d1e323293bd3ed4426ae3f688fc130ace5)


## 6.加入visual dl 观察训练
将train、val情况写入日志，并用visual dl查看训练进展

## 7.保存仍有问题不能够对比保存最佳结果